# Evals

Need predict function and evals

In [ ]:
# Reload
%load_ext autoreload
%autoreload 2


In [ ]:
%pip install -qU langsmith python-dotenv


In [ ]:
from dotenv import load_dotenv
load_dotenv()


In [ ]:
from langsmith import evaluate
from utils.langgraph import create_agent
dataset_name = "Movie Questions"

agent = create_agent()




In [ ]:

def predict(inputs: dict):
    return agent.invoke(inputs)

In [ ]:
# Eval function
from langsmith.schemas import Run, Example
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class CorrectnessScore(BaseModel):
    reasoning: str = Field(
        ..., description="Your reasoning for scoring the agent's answer as correct or incorrect")
    correct: bool = Field(..., description="If the agent's answer correct")


def correctness(run: Run, example: Example):
    judge = ChatOpenAI(model="gpt-4o-mini",
                       temperature=0).with_structured_output(CorrectnessScore)
    agent_answer = run.outputs["output"]
    correct_answer = example.outputs["output"]
    question = example.inputs["messages"][-1]["content"]

    system_prompt = f"""You are a lead QA analyst. Your job is to judge a movie data analyst's answer to users's questions. \
You will be provided with the correct answer in the form of a list of possible correct answers. You are judging what could be \
considered precision, as in as long as everything in the agent's answer is in the correct answer, it is considered correct.

For example, if the agent answers with a list of movies and their details, and the correct answer only mentions the titles, the answer is still \
considered correct if the the titles match the correct answer.

Another example is if the agent answers with a list of 3 movies and their details but the correct answer includes 10 movies. As long as the \
movies mentioned in the agent's response are included in the correct answer, the answer is still considered correct. 

If the agent did not answer with any movies, the answer is considered incorrect.


<QUESTION>
{question}
</QUESTION>

<AGENT_ANSWER>
{agent_answer}
</AGENT_ANSWER>

<CORRECT_ANSWER>
{correct_answer}
</CORRECT_ANSWER>
"""

    result: CorrectnessScore = judge.invoke(system_prompt)
    return {"key": "correctness", "score": 1 if result.correct else 0, "comment": result.reasoning}

In [ ]:
# Get current git branch
import subprocess

def get_current_branch():
    try:
        result = subprocess.run(['git', 'rev-parse', '--abbrev-ref', 'HEAD'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode != 0:
            return None
        return result.stdout.decode('utf-8').strip()
    
    except Exception as e:
        print(f"Error getting current branch: {e}")
        return "no-branch"

current_branch = get_current_branch()
print(f"Current branch: {current_branch}")


In [ ]:
from langsmith import Client
client = Client()

evaluate(predict,
         #  data=client.list_examples(dataset_name=dataset_name, splits=["sort"]),
         data=dataset_name,
         evaluators=[correctness],
         experiment_prefix=current_branch,
        #  num_repetitions=2
         )

Nice! [got my first failing evals](https://smith.langchain.com/o/d967989d-4221-53db-b0a5-665b504acba2/datasets/e1357a55-6663-4300-9695-34aad34bd544/compare?selectedSessions=6c31697f-8b1e-42e9-96c7-17ea0bbb01da)

